In [ ]:
# Modified by Bravo based on https://github.com/iAhsanJaved/FetchGoogleNews/blob/master/main.py
pip install newspaper3k

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 70.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13538 sha256=885930eddf6dc93b09bf17caa47efd9288c8c02c9229ba0d3e2d1cb8522afc55
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3340 sha256=8bce448224f9d4dd93ce662ceb3788da82e289ca1ab6ee970a1fb8893f1a0345
  Stored in directory: /root/.cache/pip/wheels/97/02/e7/a1ff1760e12bdbaab0ac824fae5c1

In [ ]:
# import the needed libraries
import requests
import pandas as pd
import time # for timing script
import xml.etree.ElementTree as ET # built in library
from datetime import datetime, timedelta
from newspaper import Article

def clean_url(search_term, data_filter):
    """
    OUTPUT : url to be fecthed for the searched_item and data_filter
     ---------------------------------------------------
    Parameters:
      today' - get headlines of the news that are released only in today
                       'this_week' - get headlines of the news that are released in this week
                       'this month' - news released in this month
                       'this_year' - news released in this year
                        number : int/str input for number of days ago
                        or '' blank to get all data
    """
    x = datetime.today()
    today = str(x)[:10]
    yesterday = str(x + timedelta(days=-1))[:10]
    this_week = str(x + timedelta(days=-7))[:10]
    if data_filter == 'today':
        time = 'after%3A' + yesterday
    elif data_filter == 'this_week':
        time = 'after%3A'+ this_week + '+before%3A' + today
    elif data_filter == 'this_year':
        time = 'after%3A'+str(x.year - 1)
    elif str(data_filter).isdigit():
        temp_time = str(x + timedelta(days=-int(data_filter)))[:10]
        time =  'after%3A'+ temp_time + '+before%3A' + today
    else:
        time=''
    url = f'https://news.google.com/rss/search?q={search_term}+'+time+'&hl=en-US&gl=US&ceid=US%3Aen'
    return url

# clear the description
def get_text(x):
    start = x.find('<p>')+3
    end = x.find('</p>')
    return x[start:end]

def get_content(url):
    """
    Extract the main content of a news article from its URL.
    """
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.text
    except Exception as e:
        return str(e)

def get_news(search_term, data_filter=None):
    """
    Search through Google News with the "search_term" and get the headlines
     and the contents of the news that was released today, this week, this month,
    or this year ("date_filter").
    """

    url = clean_url(search_term, data_filter)
    response = requests.get(url)
    # get the root directly as we have text file of string now
    root = ET.fromstring(response.text)
    #get the required data
    title = [i.text for i in root.findall('.//channel/item/title') ]
    link = [i.text for i in root.findall('.//channel/item/link') ]
    description = [i.text for i in root.findall('.//channel/item/description') ]
    pubDate = [i.text for i in root.findall('.//channel/item/pubDate') ]
    source = [i.text for i in root.findall('.//channel/item/source') ]
    # clear the description
    short_description = list(map(get_text, description))

    # extract content for each article
    content = [get_content(url) for url in link]

    # set the data frame
    df = pd.DataFrame({'title': title, 'link': link, 'description': short_description, 'date': pubDate, 'source': source, 'content': content})
    # adjust the date column
    df['date'] = pd.to_datetime(df['date'])
    # for saving purpose uncomment the below
    df.to_csv(f'{search_term}_news.csv', encoding='utf-8-sig', index=False)
    return df

if __name__ == "__main__":
    start = time.time()
    search_term = str(input('Enter your search term here: '))
    data = get_news(search_term, data_filter=5)
    end = time.time() - start
    print("Execution time", end)


Enter your search term here: Apple
Execution time 71.39073753356934
